In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

/Users/Lanado/anaconda3/envs/my_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

# Define the AlexNet Architecture
class AlexNet(nn.Module):
    def __init__(self, num_classes=100):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


In [4]:
# Data Preparation
def get_dataloaders(batch_size=64):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    train_dataset = datasets.ImageFolder(root='archive/train', transform=transform)
    valid_dataset = datasets.ImageFolder(root='archive/valid', transform=transform)
    test_dataset = datasets.ImageFolder(root='archive/test', transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, valid_loader, test_loader


In [1]:
def train_model(model, train_loader, valid_loader, device, num_epochs=10):
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters())

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0
        
        # Training loop with accuracy calculation
        for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch'):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        train_accuracy = 100 * correct_train / total_train
        print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}, Train Accuracy: {train_accuracy:.2f} %')
        
        # Validation loop with accuracy calculation
        model.eval()
        correct_valid = 0
        total_valid = 0
        running_loss_valid = 0.0
        with torch.no_grad():
            for inputs, labels in valid_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                running_loss_valid += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total_valid += labels.size(0)
                correct_valid += (predicted == labels).sum().item()

        valid_accuracy = 100 * correct_valid / total_valid
        valid_loss = running_loss_valid / len(valid_loader)
        print(f'Validation Loss: {valid_loss:.4f}, Validation Accuracy: {valid_accuracy:.2f} %')

    print('Finished Training')


In [5]:
def test_model(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f} %')

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AlexNet(num_classes=100)
train_loader, valid_loader, test_loader = get_dataloaders(batch_size=64)

In [7]:
train_model(model, train_loader, valid_loader, device, num_epochs=10)

Epoch 1/10: 100%|██████████| 211/211 [43:46<00:00, 12.45s/batch]


Epoch 1, Loss: 4.565894818418964, Train Accuracy: 1.41 %
Validation Loss: 4.4309, Validation Accuracy: 1.80 %


Epoch 2/10: 100%|██████████| 211/211 [43:53<00:00, 12.48s/batch]


Epoch 2, Loss: 4.253803169557834, Train Accuracy: 3.48 %
Validation Loss: 4.0711, Validation Accuracy: 5.00 %


Epoch 3/10: 100%|██████████| 211/211 [1:29:21<00:00, 25.41s/batch]   


...

Epoch 8, Loss: 3.1173960043920723, Train Accuracy: 21.92 %
Validation Loss: 3.0513, Validation Accuracy: 24.80 %


Epoch 9/10: 100%|██████████| 211/211 [42:20<00:00, 12.04s/batch]


Epoch 9, Loss: 2.9519548054555016, Train Accuracy: 25.32 %
Validation Loss: 3.0534, Validation Accuracy: 25.60 %


Epoch 10/10: 100%|██████████| 211/211 [37:52<00:00, 10.77s/batch]


Epoch 10, Loss: 2.799468681145618, Train Accuracy: 28.38 %
Validation Loss: 2.9846, Validation Accuracy: 24.00 %
Finished Training


In [8]:
test_model(model, test_loader, device)

Test Accuracy: 31.20 %
